In [116]:
import numpy as np
import scipy.optimize as sopt 
import matplotlib.pyplot as plt
import pydae.plot_tools as pltt
import sys
colors = pltt.set_style(plt)

In [117]:
import minionehalf

In [118]:
%matplotlib widget

In [119]:
model = minionehalf.model()
params = {'T_g':2.5,'D':2.0}
model.ini(params,0.5)
model.report_x()

p_g   =  0.80
omega =  1.00
e_cis =  0.50


In [120]:
#ssa.participation(grid).abs().round(2)
params.update({'p_l':0.8})
model.ini(params,0.5)
model.run( 1.0,{})
model.run(15.0,{'p_l':1.0})

model.post();

In [121]:
fig,axes = plt.subplots(nrows=2, sharex=True)
axes[0].plot(model.Time,  model.get_values('omega')*50,label='omega')


#axes[1].plot(grid.Time,  grid.get_values('p_e_f_5')/1e6,label='omega')

#axes.plot(grid.Time,  grid.get_values('omega_5')-grid.get_values('omega_coi'),label='omega')
#axes.plot(grid.Time,  grid.get_values('P_5'),label='P_5')

axes[0].grid()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [122]:
#model = minionehalf.model()
params = {'T_g':2.5,'D':2.0}

class optimize(minionehalf.model):
    
    def __init__(self):
        
        super().__init__()
        
        
        self.Dt_mid = 0.01
        self.Dt = 0.005


        self.P_cis_max = 0.1
        self.H_cis = 0.2
        
    def obj(self,x):
        
        Df_db = x[0]
        Df_dr = x[1]
        Dt_mid = self.Dt_mid
        
        P_cis_max = self.P_cis_max

        self.f_p = np.array([
                       [          -5.0, P_cis_max],
                       [-(Df_dr+Df_db), P_cis_max],
                       [      -(Df_db),       0.0],
                       [       (Df_db),       0.0],
                       [ (Df_dr+Df_db),-P_cis_max],
                       [           5.0,-P_cis_max]
                       ])

        
        times = np.arange(0,15,self.Dt_mid)

        params.update({'p_l':0.8, 'H':7, 'H_cis':self.H_cis, 'K_cis':1e-8})
        self.ini(params,0.5)
        enabled = 1.0
        p_l_0 = self.get_value('p_l')
        p_cis = 0.0

        for t in times:

            p_l = p_l_0
            if t>1.0: p_l = 1.0

            freq = 50*self.get_value('omega')
            e_cis = self.get_value('e_cis')
            p_cis_ref = 0.0
            if e_cis <= 0.0:
                p_cis_ref = 0.0
                p_cis = 0.0
                enabled = 0
            else:
                p_cis_ref = enabled*np.interp(freq-50,self.f_p[:,0],self.f_p[:,1])

            p_cis += Dt_mid/0.02*(p_cis_ref - p_cis)

            self.run(t,{'p_l':p_l,'p_cis':p_cis})  

        self.post();

        #axes.plot(grid.Time,  grid.get_values('omega_5')-grid.get_values('omega_coi'),label='omega')
        #axes.plot(grid.Time,  grid.get_values('P_5'),label='P_5')


        freq_min = 50*(np.min(self.get_values('omega')))
        sys.stdout.write(f'\r H_cis: {self.H_cis:5.3f}, Nadir: {freq_min:5.3f} Hz, Df_db = {Df_db:5.4f},  Df_dr = {Df_dr:5.4f}')
        sys.stdout.flush()

        nadir = 50 - freq_min
        
        self.nadir = nadir
        self.freq_min = freq_min
        
        return (1e3*nadir)**2
    
 

In [123]:
H_cis_arr = np.arange(0.02,0.5,0.01)


x_0 = np.array([0.76343571, 0.00999993]) 

opt_model_list = []
for H_cis in H_cis_arr:
    opt_model = optimize()
    
    opt_model.H_cis = H_cis
    opt_model.P_cis_max = 0.15

    
    x_0[1] = 0.01
    sol = sopt.minimize(opt_model.obj, x_0, args=(), method='Powell', tol=1e-12, bounds=[(0,1),(0.002,1)]) 
    x_0 = sol.x
    opt_model.sol = sol
    opt_model_list += [opt_model]
    # SLSQP,Powell,trust-constr,Nelder-Mead,COBYLA,BFGS

 H_cis: 0.490, Nadir: 49.420 Hz, Df_db = 0.4363,  Df_dr = 0.20328

In [141]:
models = []

for opt_model in opt_model_list:
    model = optimize()
    model.Dt = 0.001
    model.Dt_mid = 0.01
    model.H_cis = opt_model.H_cis
    model.P_cis_max = 0.15
    model.sol = opt_model.sol
    model.obj(opt_model.sol.x) 
    models += [model]



 H_cis: 0.490, Nadir: 49.420 Hz, Df_db = 0.4363,  Df_dr = 0.2032

In [162]:

fig,axes = plt.subplots(nrows=3, figsize = (3.5,6), sharex=True)

models_list = models[3:4]
colormaps = plt.cm.hot(np.linspace(0, 1, len(models_list)+10))

for i,model in enumerate(models_list):
    #for ic,df_db in enumerate([0.2,0.69,0.85]): #np.linspace(0,1.5,20): #np.array([0.1,0.71,1.0]): #

    axes[0].plot(model.Time[[0,-1]],[model.f_p[1,0]+50]*2,label='omega',lw=0.1,color = (0.8,0.8,0.8))
    axes[0].plot(model.Time[[0,-1]],[model.f_p[2,0]+50]*2,label='omega',lw=0.1,color = (0.8,0.8,0.8))

    axes[0].plot(model.Time,  model.get_values('omega')*50,label='omega',color = colormaps[i])


    
    axes[1].plot(model.Time,  model.get_values('p_cis'),label='omega',color = colormaps[i])
    axes[2].plot(model.Time,  model.get_values('e_cis'),label='omega',color = colormaps[i])

    for ax in axes:
        ax.grid()
        #ax.set_xlim(opt_model.Time[[0,-1]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [164]:
models_list[3].H_cis

IndexError: list index out of range

In [142]:
H_ciss = np.array([item.H_cis for item in models])
deadbands = np.array([item.sol.x[0] for item in models])
deltafs = np.array([item.sol.x[1] for item in models])


fig,axes = plt.subplots(nrows=1, figsize = (3.5,3.5), sharex=True)
axes.plot(H_ciss,deadbands,label='omega',color = colors[0])
axes.plot(H_ciss,deadbands,'.',label='omega',color = colors[0])

axes.plot(H_ciss,deadbands+deltafs,label='omega',color = colors[0])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [171]:
C = 4*2200e-6
V_max = 800.0
V_min = 620.0

E_c = 0.5*C*(V_max**2 - V_min**2)
E_c

1124.64

In [165]:
S_n = 20e3
H_c = 0.05
E_ref = S_n*H_c
E_ref

1000.0

In [233]:
obj(np.array([0.7 , 0.005]))

fig,axes = plt.subplots(nrows=3, figsize = (3.5,6), sharex=True)

#for ic,df_db in enumerate([0.2,0.69,0.85]): #np.linspace(0,1.5,20): #np.array([0.1,0.71,1.0]): #

axes[0].plot(model.Time,  model.get_values('omega')*50,label='omega',color = colors[ic])
axes[1].plot(model.Time,  model.get_values('p_cis'),label='omega')
axes[2].plot(model.Time,  model.get_values('e_cis'),label='omega')
    
for ax in axes:
    ax.grid()
    ax.set_xlim(times[0],times[-1])

Nadir: -0.709 Hz, Df_db = 0.700,  Df_dr = 0.005

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
l  = [0,1,2,3,4,5,6]

In [133]:
l[0:-1:2]

[0, 2, 4]